In [1]:
# --- Install ICU bindings ---
!pip install PyICU

from icu import Transliterator

# --- Define Swedish transliteration rules (simplified subset for demo) ---
rules = r"""
::NFC;
А→A; а→a;
Б→B; б→b;
В→V; в→v;
Г→G; г→g;
Д→D; д→d;
Е→E; е→e;
Ё→Jo; ё→jo;
Ж→Zj; ж→zj;
З→Z; з→z;
И→I; и→i;
Й→J; й→j;
К→K; к→k;
Л→L; л→l;
М→M; м→m;
Н→N; н→n;
О→O; о→o;
П→P; п→p;
Р→R; р→r;
С→S; с→s;
Т→T; т→t;
У→U; у→u;
Ф→F; ф→f;
Х→Ch; х→ch;
Ц→Ts; ц→ts;
Ч→Tj; ч→tj;
Ш→Sj; ш→sj;
Щ→Sjtj; щ→sjtj;
Ъ→; ъ→;
Ы→Y; ы→y;
Ь→; ь→;
Э→E; э→e;
Ю→Ju; ю→ju;
Я→Ja; я→ja;
"""

# --- Register and create Transliterator ---
trans = Transliterator.createFromRules("Russian-Latin/SV", rules, Transliterator.FORWARD)

# --- Sample words from the Swedish Wikipedia page ---
samples = [
    "Москва",       # Moskva
    "Чайковский",   # Tjajkovskij
    "Щука",         # Sjtjuka
    "Жириновский",  # Zjirinovskij
    "Юрий",         # Jurij
    "Яковлев",      # Jakovlev
    "Хрущёв",       # Chrusjtjov
    "Циолковский",  # Tsiolkovskij
]

# --- Run test ---
for w in samples:
    print(f"{w:15s} → {trans.transliterate(w)}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.6/267.6 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for PyICU: filename=pyicu-2.15.3-cp312-cp312-linux_x86_64.whl size=2720036 sha256=a2bc953ec5817526c08b87fd71139074b490edf86e5d59679606a768cc03ff7b
  Stored in directory: /root/.cache/pip/wheels/46/11/aa/9777ed706b79bd6fbe41492e7a8dbbf0699e91e0173f7be151
Successfully built PyICU


AttributeError: type object 'icu.Transliterator' has no attribute 'FORWARD'